In [86]:
from source.missing_imputation import DefiniteRegressionImputation, definite_regression_imputation
import numpy as np
from temp.missing_imputation import regression_definite_imputation as rdi
from source.base_component import extract_features, remove_outliers, intersection, union
from source.outlier_detection import cook_distance
from source.feature_selection import stepwise_feature_selection, lasso, marginal_screening
from source.pipeline import make_pipeline, make_dataset

from temp.feature_selection_si import sfs_si, lasso_si, ms_si
from temp.outlier_removal_si import cook_si

n, p = 100, 10
rng = np.random.default_rng()

dataset = []
X = rng.normal(size=(n, p))
y = rng.normal(size=(n,))
# y[:5] += 4
missing = rng.choice(list(range(n)), size=n // 10, replace=False)
y[missing] = np.nan

y_imp1 = DefiniteRegressionImputation().impute_missing(X, y)
y_imp2, cov1 = DefiniteRegressionImputation().compute_covariance(X, y, 1.0)
X, y_imp3, cov2 = rdi(X, y, 1.0)

print(np.allclose(y_imp1, y_imp2))
print(np.allclose(y_imp1, y_imp3))
print(np.allclose(cov1, cov2))


def option():
    X, y = make_dataset()
    y = definite_regression_imputation(X, y)

    O = cook_distance(X, y, 3.0)
    X, y = remove_outliers(X, y, O)

    M = marginal_screening(X, y, 5)
    X = extract_features(X, M)

    M1 = stepwise_feature_selection(X, y, 3)
    M2 = lasso(X, y, 0.08)
    M = intersection(M1, M2)
    return make_pipeline(output=M)

pipeline = option()
result = pipeline.inference(X, y, 1.0, test_index=0)[1]

print(np.allclose(y_imp1, pipeline.y))
print(np.allclose(cov1, pipeline.cov))


True
True
True
True
True


In [87]:
from source.missing_imputation import definite_regression_imputation
import numpy as np
from source.base_component import extract_features, remove_outliers, intersection, union
from source.outlier_detection import cook_distance
from source.feature_selection import stepwise_feature_selection, lasso, marginal_screening
from source.pipeline import make_pipeline, make_dataset

from temp.feature_selection_si import sfs_si, lasso_si, ms_si
from temp.outlier_removal_si import cook_si

n, p = 100, 10
rng = np.random.default_rng()

X = rng.normal(size=(n, p))

def option():
    X, y = make_dataset()
    y = definite_regression_imputation(X, y)

    O = cook_distance(X, y, 3.0)
    X, y = remove_outliers(X, y, O)

    M = marginal_screening(X, y, 5)
    X = extract_features(X, M)

    M1 = stepwise_feature_selection(X, y, 3)
    M2 = lasso(X, y, 0.08)
    M = intersection(M1, M2)
    return make_pipeline(output=M)

pipeline = option()

a = rng.normal(size=n)
b = rng.normal(size=n)
z = rng.uniform(-2, 2)
pipeline.X = X
pipeline.reset_intervals()
MO, lu = pipeline.algorithm(a, b, z)
print(*lu)

l, u = -np.inf, np.inf
M, O = list(range(p)), []
M, O, l, u = cook_si(a, b, z, X, M, O, l, u, 3.0)
M, O, l, u = ms_si(a, b, z, X, M, O, l, u, 5)
M1, O, l, u = sfs_si(a, b, z, X, M, O, l, u, 3)
M2, O, l, u = lasso_si(a, b, z, X, M, O, l, u, 0.08)
print(l, u)

-1.6303415151511693 -1.5126046275023979
-1.6303415151511693 -1.5126046275023979
